In [1]:
import pyinterp
import pyinterp.backends.xarray
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe

In [2]:
from ocn_tools._src.geoprocessing.gridding import grid_to_regular_grid

In [3]:
def test_domain():
    return dict(
       time=[pd.to_datetime('2020-01-01'), pd.to_datetime('2020-01-05')],
       lat=[40, 50],
       lon=[-60, -50],
    )

def ssh_attrs():
    return dict( units='m', long_name='Sea Surface Height',)

def time_attrs():
    return dict(long_name='Date')


def lon_attrs():
    return dict(units='degrees_east', long_name='Longitude')

def lat_attrs():
    return dict(units='degrees_north', long_name='Latitude')

def make_grid_ds(dt, dx):
    return xr.Dataset(
        coords = {
            'time': ('time', pd.date_range(*test_domain()['time'], freq=dt), time_attrs()),
            'lat': ('lat', np.arange(*test_domain()['lat'], dx), lat_attrs()),
            'lon': ('lon', np.arange(*test_domain()['lon'], dx), lon_attrs()),
        }
    ).assign(ssh=lambda ds: (ds.dims, np.ones([*ds.dims.values()]), ssh_attrs()))

In [4]:
dt = pd.to_timedelta('12H')
dx = 0.5

ds_1 = make_grid_ds(dt, dx)

In [5]:
dt = pd.to_timedelta('1D')
dx = 0.1
ds_2 = make_grid_ds(dt, dx)

In [12]:
ds_2

<xarray.Dataset>
Dimensions:  (time: 5, lat: 100, lon: 100)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * lat      (lat) float64 40.0 40.1 40.2 40.3 40.4 ... 49.5 49.6 49.7 49.8 49.9
  * lon      (lon) float64 -60.0 -59.9 -59.8 -59.7 ... -50.4 -50.3 -50.2 -50.1
Data variables:
    ssh      (time, lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

In [6]:
grid_to_regular_grid(ds_1, ds_2)

<xarray.Dataset>
Dimensions:  (time: 9, lat: 100, lon: 100)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 2020-01-01T12:00:00 ... 2020-01-05
  * lat      (lat) float64 40.0 40.1 40.2 40.3 40.4 ... 49.5 49.6 49.7 49.8 49.9
  * lon      (lon) float64 -60.0 -59.9 -59.8 -59.7 ... -50.4 -50.3 -50.2 -50.1
Data variables:
    ssh      (time, lat, lon) float64 1.0 nan nan nan nan ... nan nan nan nan
Attributes:
    regrid_method:  bilinear

In [7]:
interpolator = pyinterp.backends.xarray.Grid3D(ds_1.ssh)

In [8]:
lon_coords, lat_coords, time_coords = ds_2.lon, ds_2.lat, ds_2.time

# create meshgrid
lon, lat, time = np.meshgrid(lon_coords, lat_coords, time_coords)

ref_shape = ds_1.ssh.shape

# do interpolation
out_array = interpolator.bicubic(coords=dict(lon=lon.ravel(), lat=lat.ravel(), time=time.ravel()))

In [9]:
out_array = out_array.reshape(ref_shape)

ValueError: cannot reshape array of size 50000 into shape (9,20,20)

In [13]:
5*100*100

50000